In [1]:
import pypsa
import pandas as pd

~/opt/miniconda3/envs/pypsa-earth-old3/lib/python3.10/site-packages/pypsa/networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


In [2]:
old_netw_path = "KZ_base_main/networks/"
new_netw_path = "KZ_pypsa_vnom_fix/networks/"

n_name = "elec_s_10_ec_lcopt_Co2L-3H.nc"

# it can make sense to investigate outputs of the workflow step-by-step
# to locate the point from where the discrepancies originate
# n_name = "base.nc"
# n_name = "elec.nc"
# n_name = "elec_s.nc"
# n_name = "elec_s_10.nc"
#n_name = "elec_s_10_ec_lcopt_Co2L-3H.nc"


In [3]:
old_network = pypsa.Network(old_netw_path + n_name)
new_network = pypsa.Network(new_netw_path + n_name)

~/opt/miniconda3/envs/pypsa-earth-old3/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
~/opt/miniconda3/envs/pypsa-earth-old3/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
~/opt/miniconda3/envs/pypsa-earth-old3/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' 

## Setup comparison

Define the columns to compare across the component dataframes:

In [4]:
#gen_cols_to_compare = ["p_nom", "marginal_cost"]

gen_cols_to_compare = ["p_nom_opt"]

#['p_nom_min', 'p_nom',
#       'efficiency', 'marginal_cost', 'capital_cost', 'build_year', 'lifetime',
#       'p_nom_max', 'weight', 'control', 'type', 'p_min_pu', 'p_max_pu',
#       'p_set']

Evaluate the differences across the components:

In [5]:
gen_diff_df = (
   new_network.generators[gen_cols_to_compare] - old_network.generators[gen_cols_to_compare]
)
#lin_diff_df = (
#   new_network.lines[gen_cols_to_compare] - old_network.lines[gen_cols_to_compare] > 0
#)
#bus_diff_df = (
#   new_network.buses[gen_cols_to_compare] - old_network.buses[gen_cols_to_compare] > 0
#)

### Investigate differences in components structure

The differences are possible as PyPSA has different clustering behaviour in different versions, which also related dropping some columns

In [6]:
new_network.buses.columns.symmetric_difference(old_network.buses.columns)

Index(['lat', 'lon', 'tag_area', 'tag_substation'], dtype='object')

In [7]:
new_network.lines.columns.symmetric_difference(old_network.lines.columns)

Index(['bounds', 'country', 'dc', 'geometry', 'i_nom', 'tag_frequency',
       'tag_type', 'under_construction', 'underground', 'underwater_fraction'],
      dtype='object')

In [8]:
old_network.lines.columns.difference(new_network.lines.columns)

Index([], dtype='object')

In [9]:
new_network.lines.columns.difference(old_network.lines.columns)

Index(['bounds', 'country', 'dc', 'geometry', 'i_nom', 'tag_frequency',
       'tag_type', 'under_construction', 'underground', 'underwater_fraction'],
      dtype='object')

### Check spartial correspondance

In [10]:
old_network.buses[["x", "y"]].head(5)

,x,y
Bus,,
KZ0 0,70.847787,52.451315
KZ0 1,76.827820,43.618933
KZ0 2,65.294600,46.024153
KZ0 3,70.470225,42.780110
KZ0 4,80.181988,49.990818


In [11]:
new_network.buses[["x", "y"]].head(5)

,x,y
Bus,,
KZ0 0,70.847787,52.451315
KZ0 1,76.827820,43.618933
KZ0 2,65.294600,46.024153
KZ0 3,70.470225,42.780110
KZ0 4,80.181988,49.990818


### Check columns values

In [12]:
new_network.buses["v_nom"].unique()

array([380.,   1.])

In [13]:
new_network.lines["type"].unique()

array(['Al/St 240/40 4-bundle 380.0'], dtype=object)

In [14]:
new_network.lines["v_nom"].unique()

array([380.])

### Evaluate differences by components

In [15]:
(
    old_network.lines[["length"]].sum() 
    - new_network.lines[["length"]].sum()
)/old_network.lines[["length"]].sum()

length   -2.268877e-16
dtype: float64

In [16]:
(
    old_network.lines[["capital_cost"]].sum() 
    - new_network.lines[["capital_cost"]].sum()
)/old_network.lines[["capital_cost"]].sum()

capital_cost    0.000088
dtype: float64

In [17]:
(
    old_network.lines[["s_nom_opt"]].sum() 
    - new_network.lines[["s_nom_opt"]].sum()
)/old_network.lines[["s_nom_opt"]].sum()

s_nom_opt    2.755005e-16
dtype: float64

In [18]:
(
    old_network.lines[["s_nom"]].sum() 
    - new_network.lines[["s_nom"]].sum()
)/old_network.lines[["s_nom"]].sum()

s_nom   -1.377502e-16
dtype: float64

In [19]:
(
    old_network.lines[["s_nom"]].sum() 
    - new_network.lines[["s_nom"]].sum()
)

s_nom   -1.455192e-11
dtype: float64

Sometimes, it can be handy to compare the sums (e.g. if there are some issues with the spartial correspondance):

In [20]:
old_network.lines[["s_nom_opt"]].sum()

s_nom_opt    105639.857
dtype: float64

In [21]:
(
    old_network.lines[["s_nom_opt"]].sum() 
    - new_network.lines[["s_nom_opt"]].sum()
)/old_network.lines[["s_nom_opt"]].sum()

s_nom_opt    2.755005e-16
dtype: float64

In [22]:
(
    old_network.generators[["p_nom_opt"]].sum() 
    - new_network.generators[["p_nom_opt"]].sum()
)/old_network.generators[["p_nom_opt"]].sum()

p_nom_opt    5.248590e-08
dtype: float64

In [23]:
(
    old_network.generators[["capital_cost"]].sum() 
    - new_network.generators[["capital_cost"]].sum()
)/old_network.generators[["capital_cost"]].sum()

capital_cost    0.007295
dtype: float64

In [24]:
(
    old_network.generators[["marginal_cost"]].sum() 
    - new_network.generators[["marginal_cost"]].sum()
)/old_network.generators[["marginal_cost"]].sum()

marginal_cost    0.000008
dtype: float64

In [25]:
old_network.stores.p_set.sum() 

0.0

In [26]:
new_network.stores.p_set.sum()

0.0

In [27]:
(
    old_network.stores.e_nom_opt.sum() 
    - new_network.stores.e_nom_opt.sum()
)/old_network.stores.e_nom_opt.sum() 

-0.42572341171129463

In [28]:
old_network.stores.capital_cost.sum()

450100.0493011138

In [29]:
(
    old_network.stores.marginal_cost.sum() 
    - new_network.stores.marginal_cost.sum()
)/old_network.stores.marginal_cost.sum() 

-0.004478008285199519

In [30]:
(
    old_network.stores.capital_cost.sum() 
    - new_network.stores.capital_cost.sum()
)/old_network.stores.capital_cost.sum() 

0.0

A more detailed assesment may be needed to investigate the source of the identified differences:

In [31]:
(
    old_network.generators[old_network.generators.carrier.isin(["offwind-ac"])][["capital_cost"]]
    - new_network.generators[new_network.generators.carrier.isin(["offwind-ac"])][["capital_cost"]]
).sum()

capital_cost    15103.536874
dtype: float64